# Experimentation

So we saw that ensembling Spacy tokenized forward and backwards models worked pretty decent... let's take that further. Our model structure will be:

* Spacy
  * Forwards
  * Backwards
* SentencePiece
  * Forwards
  * Backwards
  
Let's do it in two phase, as we need some special care for sentencepiece

In [0]:
from fastai.text import *

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')

# Spacy

This first part is the exact same as from 05b

In [0]:
data_lm_spa_fwd = TextLMDataBunch.from_csv(path, 'texts.csv', text_cols='text', backwards=False, bs=16)
data_lm_spa_bwd = TextLMDataBunch.from_csv(path, 'texts.csv', text_cols='text', backwards=True, bs=16)

In [7]:
data_lm_spa_fwd.show_batch()

idx,text
0,at all .. seems like the props where taken from some bad western series of some kind ( for kids ) and they just did whatever they could with it . \n \n xxmaj what audience is this movie for ? i can only think of 12 - 14 year olds . xxmaj if you 're older than 14 you 'll have serious problems with this movie . xxmaj
1,an insult to the brain and hence completely unbelievable is that a group of 7 soldiers can kill of so many rebels without being hurt or killed themselves . xxmaj only near the end they loose a few comrades . xxmaj and that is only because they have to fight of an army of nearly 500 or more . xxmaj can you believe that ? \n \n xxmaj they
2,"to re - up for another 6 years . xxmaj her take on her time spent in the xxunk is that she did some good . xxmaj local xxmaj xxunk children got first rate medical treatment for various common xxunk not ordinarily able to afford free with an xxunk and xxunk . xxmaj her look over her shoulder at her xxmaj iraq tour was . "" xxmaj we changed some"
3,"that was xxmaj survive xxmaj style 5 + , a film that actually has meaning , combined with excellent use of scenery , cinematography , catchy dialog , funny moments , good soundtrack , excellent performances , fantastic pacing and flow . xxup fffc features the exact opposite in every way , boring scenery ( 20 minutes staring at a bland beach at night ? a completely white stage ?"
4,"xxmaj there was nothing in the bulk of the film that you could look back to and think "" xxmaj xxunk was n't that clever "" because it was n't . xxmaj in "" xxmaj fight xxmaj club "" there are flashbacks at the end showing bits where xxmaj tyler 's true identity was xxunk shown , and when you watched it again you saw more , it really was"


In [8]:
data_lm_spa_bwd.show_batch()

idx,text
0,""" getting recall n't do i but xxmaj . relationships and xxunk into steps xxunk first their through stumbling characters youthful are these , yes xxmaj . really not , well xxmaj . "" friends xxmaj "" and movie this between comparison a read i think i site this on somewhere xxmaj . change a for guys gay two the be to happen n't does just it xxmaj . movie"
1,have might it comedy more and plot a of more least at or line story different a had it if xxmaj . wall the up me drove just movie this but spencer xxmaj xxunk xxmaj like i and movies hill xxmaj xxunk xxmaj like i . movie the into minutes 5 off it turn to me drives still it and times of number them through sat have i . them
2," go a film this give to had just i , reviews bad pretty despite xxmaj xxbos . photographer ugly and fat the , character main - "" ! you have i ! ugly 're you xxmaj ! fat 're you xxmaj ! you hate i ! you hate i "" ] mirror the in herself at looking while [ \n \n . enlightenment of school own her of"
3,"but , xxunk he people the than questioning his of most to better much xxunk 've could i that feeling the get i . these of none almost on touches he xxmaj . themselves of ask should they many and folks religious of ask honestly to questions good are there , record the for xxmaj \n \n . questions his answer really to allow n't does he that groups"
4,"possesses humanity that belief his xxmaj . for stood author the that everything against goes sequences action 's it all with itself plot the xxmaj . stories his of xxunk a was that intelligence critical the possess not do , characters 's asimov xxmaj of many after named are which , characters the xxmaj . yourself for them discover and skills thinking you practice to need you xxmaj . you"


In [0]:
fwd = language_model_learner(data_lm_spa_fwd, AWD_LSTM, drop_mult=0.3).to_fp16()
bwd = language_model_learner(data_lm_spa_bwd, AWD_LSTM, drop_mult=0.3).to_fp16()

In [0]:
models = [fwd, bwd]

In [0]:
def train_models(models:list):
  bs = models[0].data.batch_size
  names = ['spa_forward', 'spa_backward']
  x = 0
  for model in models:
    lr = 1e-2
    lr *= bs/48
    model.fit_one_cycle(1, lr, moms=(0.8, 0.7))
    model.unfreeze()
    model.fit_one_cycle(1, lr/10, moms=(0.8, 0.7))
    model.save(f'{names[x]}_fine_tuned_10')
    model.save_encoder(f'{names[x]}_fine_tuned_enc_10')
    x += 1

In [14]:
train_models(models)

epoch,train_loss,valid_loss,accuracy,time
0,4.220868,3.851373,0.290625,00:09


epoch,train_loss,valid_loss,accuracy,time
0,3.901305,3.789649,0.300077,00:12


epoch,train_loss,valid_loss,accuracy,time
0,4.245296,3.881888,0.320799,00:09


epoch,train_loss,valid_loss,accuracy,time
0,3.901468,3.807374,0.328264,00:12


Note: we are only training for two epochs due to overfitting

In [0]:
df = pd.read_csv(path/'texts.csv')

In [0]:
train_df = df.iloc[:800]
valid_df = df.iloc[800:]

In [0]:
data_cls_spa_fwd = TextClasDataBunch.from_df(train_df=train_df, valid_df=valid_df, path=path, 
                                             vocab=data_lm_spa_fwd.vocab, backwards=False, bs=16)

data_cls_spa_bwd = TextClasDataBunch.from_df(train_df=train_df, valid_df=valid_df, path=path, 
                                             vocab=data_lm_spa_bwd.vocab, backwards=True, bs=16)

In [0]:
learn_cls_spa_fwd = text_classifier_learner(data_cls_spa_fwd, AWD_LSTM, drop_mult=0.5).to_fp16()
learn_cls_spa_bwd = text_classifier_learner(data_cls_spa_bwd, AWD_LSTM, drop_mult=0.5).to_fp16()

In [0]:
learn_cls_spa_fwd.load_encoder('spa_forward_fine_tuned_enc_10')
learn_cls_spa_bwd.load_encoder('spa_backward_fine_tuned_enc_10')
learn_cls_spa_fwd.freeze()
learn_cls_spa_bwd.freeze()

In [0]:
def train_models(fwd:Learner, bwd:Learner):
  models = [fwd, bwd]
  bs = fwd.data.batch_size
  names = ['forward', 'backward']
  x = 0
  for model in models:
    lr = 1e-2
    lr *= bs/48
    model.fit_one_cycle(1, lr, moms=(0.8, 0.7))
    
    model.freeze_to(-2)
    model.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7))
    
    model.freeze_to(-3)
    model.fit_one_cycle(1, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))
    
    model.unfreeze()
    model.fit_one_cycle(2, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))
  return fwd, bwd

In [26]:
learn_cls_spa_fwd, learn_cls_spa_bwd = train_models(learn_cls_spa_fwd, learn_cls_spa_bwd)

epoch,train_loss,valid_loss,accuracy,time
0,0.524549,0.529380,0.705000,00:05


epoch,train_loss,valid_loss,accuracy,time
0,0.449530,0.423241,0.810000,00:06


epoch,train_loss,valid_loss,accuracy,time
0,0.378100,0.407294,0.830000,00:08


epoch,train_loss,valid_loss,accuracy,time
0,0.302416,0.372678,0.825000,00:10
1,0.273694,0.375296,0.835000,00:10


epoch,train_loss,valid_loss,accuracy,time
0,0.592681,0.560117,0.715000,00:05


epoch,train_loss,valid_loss,accuracy,time
0,0.530375,0.538316,0.730000,00:06


epoch,train_loss,valid_loss,accuracy,time
0,0.423208,0.527028,0.750000,00:08


epoch,train_loss,valid_loss,accuracy,time
0,0.369098,0.518303,0.755000,00:11
1,0.342984,0.516153,0.770000,00:11


In [0]:
learn_cls_spa_fwd.save('cls_spa_fwd')
learn_cls_spa_bwd.save('cls_spa_bwd')

Alright! Those two are done. Onto SentencePiece

# SentencePiece

In [5]:
#we need to install sentencepiece
!pip install sentencepiece

The library's sentence piece processor is still a little bit glitchy, for now we will do the following to help us out

In [0]:
df_split = pd.DataFrame(df['text'])

In [0]:
df_split.to_csv('split.csv', index=False)

source_file = 'split.csv'
tmp_file = 'tmp.csv'
targ_file = 'data.csv'

In [0]:
string = ''

with open(source_file) as file:
  for line in file:
    for l in re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s",line):
      string += l
      string += '\n'

f = open(tmp_file,'w')
f.write(string)
f.close()

In [0]:
df_split = pd.read_csv(tmp_file, delimiter="\n", header = None, names = ['text'])

In [0]:
df_split = df_split.iloc[2:]

In [0]:
df_split.to_csv('lm_corpus.csv', index=False)

In [0]:
!bash -c 'cp /content/lm_corpus.csv /root/fastai/texts/joint_lm_corpus.csv'

In [0]:
path = Path("/root/fastai/texts/")

bs = 48
processor = SPProcessor(lang='en')
# processor = SPProcessor()

data_lm_spp_fwd = (TextList.from_csv(path, 'joint_lm_corpus.csv', processor=processor)
            .split_by_rand_pct(0.1)
            .label_for_lm()           
            .databunch(bs=bs, backwards=False))

data_lm_spp_bwd = (TextList.from_csv(path, 'joint_lm_corpus.csv', processor=processor)
            .split_by_rand_pct(0.1)
            .label_for_lm()           
            .databunch(bs=bs, backwards=True))

In [67]:
data_lm_spp_fwd.show_batch()

idx,text
0,▁kind ▁of ▁suicide ▁trip ▁has ▁his ▁career ▁been ▁on ? ▁xxmaj ▁who o sh ... ▁xxmaj ▁ ban z a i !!! ▁xxmaj ▁finally ▁this ▁was ▁directed ▁by ▁the ▁guy ▁who ▁did ▁xxmaj ▁big ▁xxmaj ▁chil l ? ▁xxmaj ▁must ▁be ▁a ▁re play ▁of ▁xxmaj ▁jonestown ▁- ▁hollywood ▁style . ▁xxmaj ▁wo o off f ! ▁xxbos ▁xxmaj ▁this ▁is ▁a ▁extremely ▁well - made ▁film . ▁xxmaj ▁the
1,"▁xxmaj ▁still , ▁a ▁fun ▁time ▁had ▁by ▁all ▁( even ▁for ▁xxmaj ▁fu ku s hi ma !) . ▁xxbos ▁xxmaj ▁this ▁movie ▁was ▁a ▁stupid ▁piece ▁of ▁crap . ▁xxmaj ▁bad ▁everything . ▁xxmaj ▁why ▁is ▁it ▁that ▁we ▁who ▁love ▁western ▁movies ▁get ▁j il ted ▁nine ▁out ▁of ▁ten ▁times ▁when ▁renting ▁western s . ▁xxmaj ▁it ▁seems ▁that ▁if ▁you ▁don ' t ▁see ▁names"
2,"▁but ▁ultimately ▁were ▁failure s ▁in ▁positive ly ▁sway ing ▁public ▁opinion . ▁xxmaj ▁so , ▁from ▁a ▁historical ▁point ▁of ▁view , ▁it ' s ▁an ▁amazing ▁and ▁sad ▁re lic ▁that ▁is ▁well ▁worth ▁seeing . ▁xxbos ▁i ▁caught ▁this ▁on ▁the ▁dis h ▁last ▁night . ▁i ▁liked ▁the ▁movie . ▁i ▁travel ed ▁to ▁xxmaj ▁russia ▁3 ▁different ▁times ▁( ad op ting ▁our ▁2 ▁kids"
3,"' . ▁xxmaj ▁the ▁plot ▁actually ▁gives ▁the ▁film ▁a ▁decent ▁base ▁- ▁or ▁at ▁least ▁more ▁of ▁a ▁decent ▁base ▁than ▁most ▁cannibal ▁films ▁- ▁and ▁it ▁follows ▁an ▁actress ▁who ▁is ▁kidnappe d ▁and ▁dragg ed ▁off ▁into ▁the ▁xxmaj ▁amazon ▁ jungle . ▁a ▁hunter ▁is ▁then ▁hired ▁to ▁find ▁her , ▁but ▁along ▁the ▁way ▁he ▁has ▁to ▁bra ve ▁the ▁natives , ▁lead ▁by ▁a"
4,"▁hunting ▁with ▁poor ▁camera ▁angles ▁ ; - ) ▁xxmaj ▁it ▁will ▁have ▁1 ▁stars ▁because ▁the ▁story ▁is ▁xxup ▁ok . ▁xxmaj ▁it ▁is ▁a ▁pity ▁that ▁xxmaj ▁ ti ▁xxmaj ▁west , ▁has ▁not ▁spent ▁more ▁time ▁to ▁review ▁his ▁story . ▁xxmaj ▁it ▁is ▁as ▁if ▁the ▁movie ▁was ▁more ▁important ▁than ▁the ▁planning . ▁xxmaj ▁because ▁you ▁have ▁a ▁camera ▁does ▁not ▁mean ▁you ▁should ▁make"


In [68]:
data_lm_spp_bwd.show_batch()

idx,text
0,". ▁it ▁to ▁tone ▁serious ▁a ▁with ▁anime ▁into ▁get ▁to ▁looking ▁anyone ▁and ), ▁so ▁think ▁to ▁reason ▁good ▁have they ▁( ▁stupid ▁is ▁anime ▁all ▁xxup ▁thinks ▁that ▁anyone , ▁anime s ▁like ▁that ▁anyone ▁for ▁great ▁is ▁series ▁this ▁that ▁saying ▁here ▁end ll ' ▁i ▁xxmaj . ▁plot ▁the ▁to al ci ru ▁c ▁is ▁which , ▁information ▁away ▁give ▁to ▁begin ▁would ▁detail ▁more ▁any"
1,"g ▁bu ▁xxmaj a ▁"" ▁xxup . ▁made ▁have ▁they ▁movies ▁animated ▁cgi ▁xxup length - ▁full ▁the ▁with ▁years ▁the ▁over ▁success ive ▁mass ▁had ▁has ▁pixar ▁xxmaj ▁xxbos . ▁it ▁liked ▁have ▁would ▁i , ▁better ▁been ▁had ▁acting ▁her ▁if ▁maybe ▁xxmaj . ▁well ▁as ▁disappointing ▁was ▁end ▁the ▁but ▁xxmaj . ▁movie ▁good ▁a ▁of s ▁making ▁the ▁has ▁script ▁the ▁of ▁part ▁first ▁the ▁xxmaj"
2,"▁to ▁close ▁stay ▁to ▁tried ▁even ▁never ▁movie ▁the ▁where ▁case ▁a ▁about ▁what ▁but ▁xxmaj ."" ▁made ▁be ▁changes ▁that ▁require ▁sometimes ▁movies , ▁medium ▁the ▁to ▁due ▁xxmaj . ▁book ▁the ▁like ▁be ▁to ▁have t ' n ▁does ▁movie the ▁"" : ▁state ▁will ▁who ) ing think ▁un . e . i ▁( ▁uncritical ▁those ▁be ▁always ▁will ▁there ▁xxmaj . ▁movie ▁this ▁about ▁anything ▁in"
3,". politics ▁ ▁not , ▁movies ▁talk ▁to ▁here re ' ▁we ▁xxmaj . ▁point ▁the side ▁be s ' ▁that ▁but ▁xxmaj . ole am ac ▁gu ▁to ▁up ed ▁add "" facts ▁"" s ' ▁clinton ▁xxmaj ▁of ▁most , ▁truth ▁in ▁when ▁xxmaj . ▁x ▁xxup ▁conclusion ▁to ▁up ▁add ▁must ▁they ▁that duce ▁de er ▁listen ▁the ▁let ▁and , d ▁ ▁xxup ▁maybe ▁and ,"
4,"s ' ▁it , en mel ▁him ▁i m ▁so <unk> s ▁ ▁xxmaj ▁see ▁go ▁xxmaj . s ▁creator ▁movie ▁outstanding ▁are ▁swedish ▁xxmaj ▁the ▁that ▁us ▁shows pollack ▁ ▁xxmaj ▁kay ▁xxmaj . ▁then ▁till ▁been ▁has ▁life ▁his ▁way ▁the ▁with ▁and ▁past ▁the ▁with ▁peace ▁make ▁to ▁trying ▁while , ▁born ▁was ▁he ▁where ▁village ▁the ▁in ▁existence ▁calm ▁a ▁create ▁to ▁tries ▁who ▁and ,"


In [0]:
fwd = language_model_learner(data_lm_spp_fwd, AWD_LSTM, drop_mult=0.3).to_fp16()
bwd = language_model_learner(data_lm_spp_bwd, AWD_LSTM, drop_mult=0.3).to_fp16()

In [0]:
models = [fwd, bwd]

SentencePiece's language model needs to be trained much longer

In [0]:
def train_models(models:list):
  bs = models[0].data.batch_size
  names = ['spp_forward', 'spp_backward']
  x = 0
  for model in models:
    lr = 1e-2
    lr *= bs/48
    model.fit_one_cycle(10, lr, moms=(0.8, 0.7))
    model.unfreeze()
    model.fit_one_cycle(5, lr/10, moms=(0.8, 0.7))
    model.save(f'{names[x]}_fine_tuned_10')
    model.save_encoder(f'{names[x]}_fine_tuned_enc_10')
    x += 1

Also, unlike our previous model, the SentencePiece models don't quite achieve as high as accuracy as the others do, and 20% looks to be close enough

In [100]:
train_models(models)

epoch,train_loss,valid_loss,accuracy,time
0,7.804316,6.360443,0.068229,00:04
1,6.428554,5.818271,0.114323,00:04
2,5.731979,5.327805,0.164993,00:04
3,5.256244,5.115634,0.179948,00:04
4,4.917917,5.015893,0.188356,00:04
5,4.654929,4.972468,0.190290,00:04
6,4.424974,4.951478,0.194940,00:04
7,4.241974,4.949923,0.196801,00:04
8,4.103992,4.955640,0.196354,00:04
9,4.025286,4.960266,0.196429,00:04


epoch,train_loss,valid_loss,accuracy,time
0,3.968590,4.956451,0.197879,00:05
1,3.891594,4.921497,0.207701,00:05
2,3.691188,4.934330,0.207180,00:05
3,3.428648,4.995906,0.207478,00:05
4,3.216956,5.061259,0.205060,00:05


epoch,train_loss,valid_loss,accuracy,time
0,7.653606,6.233237,0.111177,00:04
1,6.280353,5.733878,0.166336,00:04
2,5.561140,5.319937,0.197652,00:04
3,5.098863,5.154798,0.211243,00:04
4,4.762967,5.085770,0.219279,00:04
5,4.486774,5.071057,0.222652,00:04
6,4.237877,5.068944,0.224438,00:04
7,4.042550,5.083763,0.226852,00:04
8,3.902139,5.094715,0.226422,00:04
9,3.822707,5.097179,0.226620,00:04


epoch,train_loss,valid_loss,accuracy,time
0,3.741575,5.154630,0.228902,00:05
1,3.662651,5.210241,0.232110,00:05
2,3.416126,5.222067,0.233829,00:05
3,3.108026,5.293548,0.233631,00:05
4,2.873368,5.337930,0.231316,00:05


Oddly enough, our backwards model has higher accuracy! In any case, onto the next step!

In [0]:
df = pd.read_csv(path/'texts.csv')

In [0]:
bs = 16

data_cls_spp_fwd = (TextList.from_df(df, processor=processor, cols=1, path=path, vocab=data_lm_spp_fwd.vocab)
             .split_from_df()
             .label_from_df(cols=0, classes=['negative', 'positive'])
             .databunch(bs=bs))

data_cls_spp_bwd = (TextList.from_df(df, processor=processor, cols=1, path=path, vocab=data_lm_spp_bwd.vocab)
             .split_from_df()
             .label_from_df(cols=0, classes=['negative', 'positive'])
             .databunch(bs=bs, backwards=True))

In [89]:
data_cls_spp_fwd.show_batch()

text,target
"▁xxbos ▁xxmaj ▁ra is ing ▁xxmaj ▁vic tor ▁xxmaj ▁ va r ga s : ▁a ▁xxmaj ▁review ▁xxmaj ▁you ▁know , ▁xxmaj ▁ra is ing ▁xxmaj ▁vic tor ▁xxmaj ▁ va r ga s ▁is ▁like ▁stick ing ▁your ▁hands ▁into ▁a ▁big , ▁steam ing ▁bowl ▁of ▁o at me al . ▁xxmaj ▁it ' s ▁warm ▁and ▁go o ey , ▁but ▁you ' re ▁not ▁sure",negative
"▁xxbos ▁xxup ▁the ▁xxup ▁shop ▁xxup ▁around ▁xxup ▁the ▁xxup ▁corner ▁is ▁one ▁of ▁the ▁sweet est ▁and ▁most ▁feel - good ▁romantic ▁comedies ▁ever ▁made . ▁xxmaj ▁there ' s ▁just ▁no ▁getting ▁around ▁that , ▁and ▁it ' s ▁hard ▁to ▁actually ▁put ▁one ' s ▁feeling ▁for ▁this ▁film ▁into ▁words . ▁xxmaj ▁it ' s ▁not ▁one ▁of ▁those ▁films ▁that ▁tries ▁too ▁hard , ▁nor",positive
"▁xxbos ▁xxmaj ▁now ▁that ▁xxmaj ▁che (200 8) ▁has ▁finished ▁its ▁relative ly ▁short ▁xxmaj ▁ australia n ▁cinema ▁run ▁( extremely ▁limited ▁release : 1 ▁screen ▁in ▁xxmaj ▁sy d ney , ▁after ▁6 w k s ), ▁i ▁can ▁guilt less ly ▁join ▁both ▁host s ▁of ▁"" at ▁xxmaj ▁the ▁xxmaj ▁movies "" ▁in ▁taking ▁xxmaj ▁steven ▁xxmaj ▁soderbergh ▁to ▁task . ▁xxmaj ▁it ' s ▁usually",negative
▁xxbos ▁xxmaj ▁this ▁film ▁sat ▁on ▁my ▁xxmaj ▁ ti v o ▁for ▁week s ▁before ▁i ▁watched ▁it . ▁i ▁ d read ed ▁a ▁self - indulge nt ▁ y up p ie ▁flick ▁about ▁relationships ▁gone ▁bad . ▁i ▁was ▁wrong ; ▁this ▁was ▁an ▁en g ro s s ing ▁ex cu rs ion ▁into ▁the ▁sc rew ed - up ▁ li bi do s,positive
"▁xxbos ▁xxmaj ▁many ▁neglect ▁that ▁this ▁is n ' t ▁just ▁a ▁classic ▁due ▁to ▁the ▁fact ▁that ▁it ' s ▁the ▁first ▁xxup ▁3 d ▁game , ▁or ▁even ▁the ▁first ▁shoot - ' em - up . ▁xxmaj ▁it ' s ▁also ▁one ▁of ▁the ▁first ▁steal th ▁games , ▁one ▁of ▁the ▁only ( and ▁definite ly ▁the ▁first ) ▁truly ▁claustrophobic ▁games , ▁and ▁just ▁a",positive


In [90]:
data_cls_spp_bwd.show_batch()

text,target
". s t ▁gri ▁of ▁bowl ▁a ▁into ▁hands ▁my ▁stick ▁go ▁to ▁going m ' ▁i ▁xxmaj , ▁me ▁excuse ll ' ▁you ▁if ▁so ▁xxmaj ? ▁him ▁about ▁film ▁a ▁like ▁i ▁can ▁how ▁ultimately ▁so , ▁him ▁like t ' ▁don ▁i ▁way ▁either ▁xxmaj ' ? en che - h ▁ma v ▁lu ▁my ▁into ▁get ▁to ▁want ▁ju , e be - bay ▁ y",negative
". ▁could ▁possibly ▁very ▁we ▁that ▁feeling ▁it ▁from ▁away ▁come ▁we ▁that ▁is ▁film ▁this ▁about ▁thing ▁best ▁the ; ▁film ▁this ▁in ed ▁depict ▁romance ▁the ▁have ▁to ▁as ▁lucky ▁so ▁be ▁all ▁should ▁we ▁xxmaj . ▁role ▁his ▁in e ▁impeccabl ▁is ▁stewart ▁xxmaj ▁and , ▁charming , ▁funny , ▁sweet s ' ▁it ▁xxmaj . ▁instead ▁christmas ▁xxmaj ▁this ▁corner ▁xxup ▁the ▁xxup ▁around ▁xxup ▁shop",positive
") ▁10 ▁/ 2 ( . ▁subject ▁the ▁about ring tte ma s ▁ ▁a ▁just ▁even ▁know ▁who ▁all ▁of s ▁heart ▁the ▁break ▁to d n bou ▁ ▁is ▁it ▁xxmaj . ▁costs ▁all ▁at ▁avoided ▁be ▁to , ▁film ▁soderbergh ▁xxmaj ▁a ▁for , ingly ock sh -- ery ▁mis ▁irritating , ▁confusing ▁just ▁is ite ver ▁ ▁cinema able in m ▁inter ▁this ▁of ▁rest ▁the",negative
". ▁end ▁xxup ▁the ▁xxup . ▁other ▁each ▁at ▁look ▁they ▁xxmaj . ▁window ▁the ▁down s ▁roll , ▁up s ▁drive ▁movie ▁the ▁in ▁earlier ▁her ▁pay t ' n ▁did ▁who or ▁contract ▁the ▁xxmaj . ▁business ▁does ▁normally ▁she ▁where ▁corner ▁the ▁at ing ▁arriv ▁and , ▁street ▁the ▁down ▁walking ▁prostitute ▁the ▁see ▁we ▁and , ▁off ▁goes ▁gun ▁the , ▁eventually ▁xxmaj . im ▁a",positive
"▁10 ▁/ ▁8 . ▁games d ▁3 ▁xxup ▁of ▁fans ▁all ▁to ▁this ▁recommended ▁i . ▁appeared ▁first ▁he ▁after ▁years een ▁fift ▁nearly ?) ▁especially ▁even ▁maybe or ( ▁even , ▁recognition ▁the ▁deserves ▁he ... s ▁shoe s ' . j . ▁b ▁into ▁step ▁and ▁bunker ▁the ▁enter ▁to ▁door ▁the ▁open , ger ▁lu ▁xxmaj ▁the ▁up ▁load ▁so ▁xxmaj . ▁one ▁this ▁to ▁existence ▁their",positive


Let's make our learners and train!

In [0]:
path_m = Path('/root/fastai/texts/models')

In [0]:
learn_cls_spp_fwd = text_classifier_learner(data_cls_spp_fwd, AWD_LSTM, drop_mult=0.5).to_fp16()
learn_cls_spp_bwd = text_classifier_learner(data_cls_spp_bwd, AWD_LSTM, drop_mult=0.5).to_fp16()
learn_cls_spp_fwd.load_encoder(path_m/'spp_forward_fine_tuned_enc_10')
learn_cls_spp_bwd.load_encoder(path_m/'spp_backward_fine_tuned_enc_10')
learn_cls_spp_fwd.freeze()
learn_cls_spp_bwd.freeze()

In [0]:
def train_models(list:models):
  bs = models[0].data.batch_size
  for model in models:
    lr = 2e-2
    lr *= bs/48
    model.fit_one_cycle(1, lr, moms=(0.8,0.7))
    
    model.freeze_to(-2)
    model.fit_one_cycle(1, slice(lr/(2.6**4), lr), moms=(0.8,0.7))
    
    model.freeze_to(-3)
    model.fit_one_cycle(1, slice(lr/2/(2.6**4), lr/2), moms=(0.8,0.7))
    
    model.unfreeze()
    model.fit_one_cycle(2, slice(lr/10/(2.6**4), lr/10), moms=(0.8, 0.7))
  return models

In [0]:
models = [learn_cls_spp_fwd, learn_cls_spp_bwd]

In [109]:
learn_cls_spp_fwd, learn_cls_spp_bwd = train_models(models)

epoch,train_loss,valid_loss,accuracy,time
0,0.662650,0.621215,0.685000,00:06


epoch,train_loss,valid_loss,accuracy,time
0,0.630213,0.673570,0.685000,00:07


epoch,train_loss,valid_loss,accuracy,time
0,0.558474,0.609428,0.675000,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.470216,0.619855,0.660000,00:12
1,0.447875,0.640083,0.640000,00:12


epoch,train_loss,valid_loss,accuracy,time
0,0.629664,0.618361,0.645000,00:06


epoch,train_loss,valid_loss,accuracy,time
0,0.609980,0.605668,0.680000,00:07


epoch,train_loss,valid_loss,accuracy,time
0,0.573174,0.576597,0.705000,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.501281,0.571550,0.735000,00:12
1,0.484979,0.562025,0.705000,00:12


Notice we still see that strange behavior, the backwards model outperformed the forwards model! In any case, time for an ensemble again!

In [0]:
preds_a, targs_a = learn_cls_spa_fwd.get_preds(ordered=True)
preds_b, targs_b = learn_cls_spa_bwd.get_preds(ordered=True)
preds_c, targs_c = learn_cls_spp_fwd.get_preds(ordered=True)
preds_d, targs_d = learn_cls_spp_bwd.get_preds(ordered=True)

In [111]:
accuracy(preds_a, targs_b) == accuracy(preds_a, targs_a)

tensor(1, dtype=torch.uint8)

In [112]:
accuracy(preds_a, targs_d) == accuracy(preds_a, targs_a)

tensor(1, dtype=torch.uint8)

In [0]:
preds_spa = (preds_a + preds_b) /2
preds_spp = (preds_c + preds_d) /2

In [0]:
preds_avg = (preds_spa + preds_spp)/2

In [120]:
accuracy(preds_avg, targs_a)

tensor(0.8000)

Couldn't quite beat it.... odd. Perhaps something wen't wrong? Perhaps we need to actually use the entire dataset. I leave that to you all to run for yourselves and see how you do, you have most of the code already!

In [121]:
accuracy(preds_spa, targs_a)

tensor(0.8400)

In [122]:
accuracy(preds_spp, targs_a)

tensor(0.6950)

Notice our SentencePiece model did not do nearly as well. Perhaps we need to fit our language model for longer.